<a href="https://colab.research.google.com/github/STYLO009/NLP-Natural-Language-Processing-/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('train.txt',sep = ';', header=None, names=['text','emotions'])

In [3]:
df.head()

,text,emotions
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [5]:
unique_emotions = df['emotions'].unique()
emotions = {}
i = 0
for emotion in unique_emotions:
    emotions[emotion] = i
    i += 1
df['emotions'] = df['emotions'].map(emotions)

In [6]:
df

,text,emotions
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,1
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,1
...,...,...
15995,i just had a very brief time in the beanbag an...,0
15996,i am now turning and i feel pathetic that i am...,0
15997,i feel strong and good overall,5
15998,i feel like this was such a rude comment and i...,1


In [7]:
df['text'] = df['text'].apply(lambda x : x.lower())

In [8]:
import string

def remove_punc(txt):
  return txt.translate(str.maketrans('', '', string.punctuation))

In [9]:
df['text'] = df['text'].apply(remove_punc)

In [30]:
def remove_num(txt):
  new = ""
  for i in txt:
    if not i.isdigit():
      new += i
  return new

In [31]:
def remove_emojis(txt):
  new = ""
  for i in txt:
    if i.isascii():
      new += i
  return new

In [20]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [21]:
stop_words = set(stopwords.words('english'))
len(stop_words)

198

In [23]:
def remove(txt):
  word_tokens = word_tokenize(txt)
  cleaned = []
  for i in word_tokens:
    if i not in stop_words:
      cleaned.append(i)

  return ' '.join(cleaned)

In [34]:
df['text'] = df['text'].apply(remove)

In [39]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text'],df['emotions'],test_size=0.3,random_state=42)

In [40]:
from tkinter.constants import X
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

vector = CountVectorizer()
X_train = vector.fit_transform(X_train)
X_test = vector.transform(X_test)

In [41]:
nb = MultinomialNB()
nb.fit(X_train,y_train)

MultinomialNB()

In [42]:
pred = nb.predict(X_test)

In [44]:
pred_bow = nb.predict(X_test)

In [45]:
accuracy_score(y_test,pred)

0.7672916666666667

In [47]:
from sklearn.model_selection import train_test_split

# Re-split the data to get the original text versions for TF-IDF
X_train_text, X_test_text, y_train_tfidf, y_test_tfidf = train_test_split(df['text'],df['emotions'],test_size=0.3,random_state=42)

vectors = TfidfVectorizer()
X_train_v = vectors.fit_transform(X_train_text)
X_test_v = vectors.transform(X_test_text)

In [49]:
nb2 = MultinomialNB()
nb2.fit(X_train_v,y_train_tfidf)

MultinomialNB()

In [50]:
pred2 = nb2.predict(X_test_v)
pred_tfidf = nb2.predict(X_test_v)

In [51]:
accuracy_score(y_test_tfidf,pred2)

0.6622916666666666

In [53]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train_v, y_train_tfidf)

pred_log_reg = log_reg.predict(X_test_v)

accuracy_log_reg = accuracy_score(y_test_tfidf, pred_log_reg)
print(f"Logistic Regression Accuracy: {accuracy_log_reg}")

Logistic Regression Accuracy: 0.8516666666666667
